In [1]:
import os
import requests
from pytubefix import YouTube
import sys
import time
import re
from pprint import pprint

In [3]:
def youtube_download(url, path="."):
    yt = YouTube(url=url)
    print(yt.title)

    dl_yt_thumbnail(yt)

    dl_yt_audio_mp3(yt)

    return 0


def dl_yt_thumbnail(yt: YouTube):
    thumbnail_url = yt.thumbnail_url

    thumbnail_response = requests.get(thumbnail_url)

    if thumbnail_response.status_code == 200:
        thumbnail_file = os.path.join(".", "thumbnail.jpg")
        with open(thumbnail_file, "wb") as f:
            f.write(thumbnail_response.content)
        print("Thumbnail saved!")
    else:
        print(f"Error: {thumbnail_response.status_code}")


def dl_yt_audio_mp3(yt: YouTube):
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_dl = yt.streams.get_audio_only()
    audio_dl.download(filename="audio_test", mp3=True)

In [4]:
youtube_url = "https://www.youtube.com/watch?v=3lasolbFYVM&t=1146s"

In [5]:
youtube_download(youtube_url)

Kiki Solvej techno & rave live @ the Bahamas
Thumbnail saved!


0

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [51]:
from openai import OpenAI

In [53]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [58]:
stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

This is a test.

In [59]:
from openai.types.audio import (
    Transcription,
    TranscriptionSegment,
    TranscriptionVerbose,
    TranscriptionWord,
    TranscriptionCreateResponse,
)

In [63]:
from pathlib import Path

In [66]:
pprint(client.models.list())

SyncPage[Model](data=[Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='chatgpt-4o-latest', created=1723515131, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-3.5-turbo-16k', created=168375

In [69]:
transcription_response = client.audio.transcriptions.create(
    file=Path("audio_test.mp3"),
    language="en",
    model="whisper-1",
)

In [70]:
transcription_response.text

"It's the first time ever I order Uber Eats this early. But you see I'm a bit tight on time because I have to pack. Apparently I'm going to Jackson Hole for a Polestar campaign and it's got nothing to do with this video but this being the 20th time I leave this year I figured I would show you my perfect minimal flying everyday carry bag. I've been on so many flights this year and to be honest I've developed this obsession with how to make my travels easier on me which is why I literally came up with a minimal flying everyday carry bag I can bring 99% of items from Amazon but this to me is useful whenever I decide to transport my camera and a backpack. Truthfully I really do wish I could bring you guys all the time to these type of events. Let's take a look at what I packed to get here and how my flight time went. So this here is codename 154 City Pack. The only item here that doesn't sell on Amazon. I just recently brought this with me to Berlin. I've gotta say it's incredibly well sor

In [71]:
summary_response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=[
        {"role": "user", "content": "Summarize the following text:"},
        {"role": "user", "content": transcription_response.text},
    ],
    stream=True,
)

In [74]:
for chunk in summary_response:
    print(chunk.choices[0].delta.content or "", end="")

The text describes the author's experience preparing for a trip, specifically focusing on their minimal everyday carry (EDC) kit designed for hassle-free flights. The author explains that they've been on many flights this year and have developed a preference for simple and efficient packing. They detail various items they carry, especially highlighting a minimal Italian leather bag they use — the 154 City Pack — which is not available on Amazon but has great capacity and thoughtful design. Alongside the bag, the author discusses electronics like their Pixel 9 Pro XL phone (praising its display, battery life, and camera), accessories like a Caseology phone case and MOFT wallet stand, and travel essentials such as the Bose QuietComfort Ultra headphones, X-Real AR glasses for immersive media consumption, and an Anker 3-in-1 power bank. The text emphasizes the practicality of these items in making travel more convenient, with a focus on lightweight, versatile gear often available on Amazon

In [8]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

c:\Code\yt_whisper_streamlit\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
torch.cuda.is_available()

True

In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
)

In [11]:
result = pipe("audio_test.mp3")

c:\Code\yt_whisper_streamlit\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
c:\Code\yt_whisper_streamlit\.venv\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:599: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Passing a tuple of `past_key_v

In [12]:
pprint(result["text"])

(' Land und Precht Schönen guten Morgen Richard. Guten Morgen Markus. Heute '
 'klinge ich selber ein bisschen so wie du. Ich habe es mir jetzt auch '
 'eingefangen. Kam irgendwie über die Leitung, muss das gekommen sein. Ja, '
 'siehst du es. Es gibt inzwischen fließende Grenzen zwischen Viren und '
 'Computerviren. Ja, zumindest da sind wir jetzt offenbar digital. Das '
 'funktioniert. Aber erstmal die Frage, wie geht es dir denn, Richard? Geht es '
 'dir gut? Wo erreiche ich dich? Was treibst du gerade? Ja, also ich habe '
 'diesen Virusquatsch da seit drei Wochen. Aber inzwischen geht es mir '
 'eigentlich ganz gut. Ich bin auch guter Dinge. Ich halte hier gerade die '
 'Neuausgabe von Wer bin ich? Ich habe das ja nach ewig langer Zeit, nach 15 '
 'Jahren mal überarbeitet, aktualisiert. Wer bin ich und wenn ja, wie viele? '
 'Ja, genau. Wer bin ich und wenn ja, wie viele? Ach Quatsch. Ich meine, das '
 'Buch ist geschrieben 2006. Und da waren natürlich auch Zahlen drin und '
 'Erke

In [13]:
import ollama

In [14]:
system_prompt = "Summarize the following transcript from a youtube video: '"
system_prompt += result["text"]

In [15]:
response = ollama.chat(
    model="llama3.1", messages=[{"role": "user", "content": system_prompt}]
)

In [16]:
pprint(response)

{'created_at': '2024-10-04T11:01:50.7031274Z',
 'done': True,
 'done_reason': 'stop',
 'eval_count': 335,
 'eval_duration': 36572374000,
 'load_duration': 14922138400,
 'message': {'content': 'Es tut mir leid, aber die Anfrage enthält eine '
                        'Audio-Datei oder einen anderen Toninhalt, der nicht '
                        'als Text interpretiert werden kann. Ich kann Ihnen '
                        'jedoch eine Zusammenfassung und eine Analyse des '
                        'Inhalts bieten.\n'
                        '\n'
                        'Der Interviewende Markus bewundert Richard von '
                        'Weizsäckers Fähigkeit, den Wandel in den Köpfen zu '
                        'erkennen und die positiven Aspekte davon '
                        'hervorzuheben. Er findet es interessant, wie die '
                        'Skandinavier ihre Kultur und Gesellschaft '
                        'weiterentwickeln, indem sie "nudging" (Schubsen) '
           

In [17]:
pprint(response["message"]["content"])

('Es tut mir leid, aber die Anfrage enthält eine Audio-Datei oder einen '
 'anderen Toninhalt, der nicht als Text interpretiert werden kann. Ich kann '
 'Ihnen jedoch eine Zusammenfassung und eine Analyse des Inhalts bieten.\n'
 '\n'
 'Der Interviewende Markus bewundert Richard von Weizsäckers Fähigkeit, den '
 'Wandel in den Köpfen zu erkennen und die positiven Aspekte davon '
 'hervorzuheben. Er findet es interessant, wie die Skandinavier ihre Kultur '
 'und Gesellschaft weiterentwickeln, indem sie "nudging" (Schubsen) anwenden, '
 'um das Verhalten ihrer Bürger zu beeinflussen.\n'
 '\n'
 'Markus meint, dass Deutschland zu groß ist, um noch alle Probleme lösen zu '
 'können. Er erwähnt ein Buch von einem Schweizer über den Charme kleiner '
 'Länder und fragt sich, ob dies nicht auch für Deutschland gelten könnte.\n'
 '\n'
 'Der Interviewende Richard von Weizsäcker schließt mit einer positiven Note '
 'ab, indem er sagt: "Hoffnung ist kein Gefühl. Hoffnung ist eine Pflicht." Er '
 'be